# Use IVIS to reduce the dimensionality of image data

In [ ]:
!pip install ivis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.1-cp38-cp38-linux_x86_64.whl size=396888 sha256=b88e2aa24d2043fc2bc23041db23df697fab04505ddb5fe5005a1f59893c44a6
  Stored in directory: /root/.cache/pip/wheels/f9/93/19/30511c4a9ae6b4937455a134c34a39e13943e2c6f46fcd2ed2
Successfully built annoy


In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
# Ivis uses several stochastic processes.
# To make sure that results are consistent from one run to another,
# we fix the random seed

import os
os.environ["PYTHONHASHSEED"]="1234"

import random
import numpy as np

np.random.seed(1234)
random.seed(1234)
tf.random.set_seed(1234)

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.1 MB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/DL_images/temp_vs_trop/model_dev/'

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
pilot_tt = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
pilot_tt = pilot_tt.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())


dls = pilot_tt.dataloaders(path)

In [ ]:
cd '/content/drive/MyDrive/DL_images/temp_vs_trop/model_dev/'

/content/drive/MyDrive/DL_images/temp_vs_trop/model_dev


## Load in the model that is trained to classify specimens into one of four groups

In [ ]:
learn_inf = load_learner('prunus_res50_temp_trop_v3_fp16_24epochs.pkl')


In [ ]:
learn_inf.model.cuda()

### These are the four groups:

In [ ]:
learn_inf.dls.vocab

['neotropical_racemose', 'paleotropical_racemose', 'temperate_diploid', 'temperate_racemose']

### Now define the hooks to get features from inner network layers

In [ ]:
class Hook():
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_func)
    def hook_func(self, m, i, o): self.stored = o.detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()

In [ ]:
class HookBwd():
    def __init__(self, m):
        self.hook = m.register_backward_hook(self.hook_func)
    def hook_func(self, m, gi, go): self.stored = go[0].detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()


In [ ]:
import pandas as pd

## The strategy is to load in images from specimens known to belong to each of the four categories

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/neotropical_racemose

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/neotropical_racemose


In [ ]:
img_holder = os.listdir()

In [ ]:
len(img_holder)

100

In [ ]:
learn_inf.model

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_racemose

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_racemose


In [ ]:
img_holder_temp_race = os.listdir()

In [ ]:
len(img_holder_temp_race)

100

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_diploid

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_diploid


In [ ]:
img_holder_temp_dipl = os.listdir()

In [ ]:
len(img_holder_temp_dipl)

100

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/neotropical_racemose/

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/neotropical_racemose


In [ ]:
img_holder_trop_neo = os.listdir()

In [ ]:
len(img_holder_trop_neo)

100

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/paleotropical_racemose/

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/paleotropical_racemose


In [ ]:
img_holder_trop_pal = os.listdir()

In [ ]:
len(img_holder_trop_pal)

100

In [ ]:
img_holder_trop_pal

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve


### Get only representatives from the 400 total that are classified correctly with high accuracy

In [ ]:
filtered_df = pd.read_csv("test100_set.csv")

In [ ]:
paleo_race_imgs_df = filtered_df[filtered_df.classification == "paleotropical_racemose"]

In [ ]:
pr = paleo_race_imgs_df['image'].to_list()

In [ ]:
len(pr)

97

In [ ]:
neo_race_imgs_df = filtered_df[filtered_df.classification == "neotropical_racemose"]

In [ ]:
nr = neo_race_imgs_df['image'].to_list()

In [ ]:
len(nr)

65

In [ ]:
temp_race_imgs_df = filtered_df[filtered_df.classification == "temperate_racemose"]

In [ ]:
tr = temp_race_imgs_df['image'].to_list()

In [ ]:
len(tr)

77

In [ ]:
temp_dipl_imgs_df = filtered_df[filtered_df.classification == "temperate_diploid"]

In [ ]:
td = temp_dipl_imgs_df['image'].to_list()

In [ ]:
len(td)

96


# The loops to get vectors from model for those strongly classified



### Temperate racemose

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_racemose

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_racemose


In [ ]:
act_hold = []
act_hold2 = []
for counter in range(0,100):
  img = tr[counter];
  x, = first(dls.test_dl([img]));
  with Hook(learn_inf.model[1]) as hook:
      with torch.no_grad(): output = learn_inf.model.eval()(x.cuda())
      act = hook.stored
#  cls = 1
  with HookBwd(learn_inf.model[1][6]) as hookg:
      with Hook(learn_inf.model[1][6]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
#  print("act: \n", act[0])
  act_hold.append(act)
  act_hold2.append(act.cpu().detach().numpy())


In [ ]:
act_temp_race = np.asarray(act_hold2)

In [ ]:
act_temp_race

array([[[ 0.9333843 , -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.32144293, -0.7372962 ]],

       [[ 0.95625395, -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]],

       [[ 0.58536696, -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]],

       ...,

       [[-0.6916634 , -0.5712142 ,  0.13919507, ..., -0.26591718, -0.29433134,  0.00665196]],

       [[-0.6725931 , -0.09186503, -0.14084442, ..., -0.7209007 ,  1.69959   , -0.7372962 ]],

       [[ 2.794887  , -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]]], dtype=float32)

### Temperate diploid

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_diploid

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_diploid


In [ ]:
act_hold = []
act_hold2 = []
for counter in range(0,100):
  img = td[counter];
  x, = first(dls.test_dl([img]));
  with Hook(learn_inf.model[1]) as hook:
      with torch.no_grad(): output = learn_inf.model.eval()(x.cuda())
      act = hook.stored
#  cls = 1
  with HookBwd(learn_inf.model[1][6]) as hookg:
      with Hook(learn_inf.model[1][6]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
#  print("act: \n", act[0])
  act_hold.append(act)
  act_hold2.append(act.cpu().detach().numpy())


In [ ]:
act_temp_dipl = np.asarray(act_hold2)

In [ ]:
act_temp_dipl

array([[[-0.6916634 , -0.5712142 , -0.20737268, ..., -0.7209007 ,  2.1842215 , -0.476812  ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  0.3786743 , -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  2.3477604 ,  1.0534972 ]],

       ...,

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  0.74480754, -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  1.4509704 , -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  0.9980766 , -0.7372962 ]]], dtype=float32)

### Neotropical racemose

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/neotropical_racemose

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/neotropical_racemose


In [ ]:
act_hold = []
act_hold2 = []
for counter in range(0,100):
  img = nr[counter];
  x, = first(dls.test_dl([img]));
  with Hook(learn_inf.model[1]) as hook:
      with torch.no_grad(): output = learn_inf.model.eval()(x.cuda())
      act = hook.stored
#  cls = 1
  with HookBwd(learn_inf.model[1][6]) as hookg:
      with Hook(learn_inf.model[1][6]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
#  print("act: \n", act[0])
  act_hold.append(act)
  act_hold2.append(act.cpu().detach().numpy())


In [ ]:
act_trop_neo = np.asarray(act_hold2)

In [ ]:
act_trop_neo

array([[[-0.49877098, -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  1.0083547 ,  0.06104004]],

       [[-0.4815011 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  0.7355752 , -0.7372962 ]],

       ...,

       [[ 0.72521627, -0.5712142 , -0.10812803, ..., -0.7209007 , -0.65128195, -0.7372962 ]],

       [[ 0.9805852 , -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.61317545, -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]]], dtype=float32)

### Paleotropical racemose

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/paleotropical_racemose

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/paleotropical_racemose


In [ ]:
act_hold = []
act_hold2 = []
for counter in range(0,100):
  img = pr[counter];
  x, = first(dls.test_dl([img]));
  with Hook(learn_inf.model[1]) as hook:
      with torch.no_grad(): output = learn_inf.model.eval()(x.cuda())
      act = hook.stored
#  cls = 1
  with HookBwd(learn_inf.model[1][6]) as hookg:
      with Hook(learn_inf.model[1][6]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
#  print("act: \n", act[0])
  act_hold.append(act)
  act_hold2.append(act.cpu().detach().numpy())


In [ ]:
act_trop_pal = np.asarray(act_hold2)

In [ ]:
act_trop_pal

array([[[ 0.15426646, -0.52265835, -0.5788494 , ..., -0.55679685, -0.6570689 , -0.7372962 ]],

       [[ 0.06092159,  1.1801544 , -0.5788494 , ..., -0.7209007 , -0.6525868 , -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  0.39502937, -0.7372962 ]],

       ...,

       [[ 0.7112668 , -0.01032911, -0.5788494 , ..., -0.6430118 , -0.6570689 , -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]],

       [[ 1.6722367 , -0.01085315, -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]]], dtype=float32)

# All 100

### Temperate Racemose

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_racemose

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_racemose


In [ ]:
act_hold = []
act_hold2 = []
for counter in range(0,100):
  img = img_holder_temp_race[counter];
  x, = first(dls.test_dl([img]));
  with Hook(learn_inf.model[1]) as hook:
      with torch.no_grad(): output = learn_inf.model.eval()(x.cuda())
      act = hook.stored
#  cls = 1
  with HookBwd(learn_inf.model[1][6]) as hookg:
      with Hook(learn_inf.model[1][6]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
#  print("act: \n", act[0])
  act_hold.append(act)
  act_hold2.append(act.cpu().detach().numpy())


In [ ]:
act_temp_race = np.asarray(act_hold2)

In [ ]:
act_temp_race

array([[[ 0.9333843 , -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.32144293, -0.7372962 ]],

       [[ 0.95625395, -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]],

       [[ 0.58536696, -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]],

       ...,

       [[-0.6725931 , -0.09186503, -0.14084442, ..., -0.7209007 ,  1.69959   , -0.7372962 ]],

       [[ 0.4219094 , -0.5712142 ,  0.6261389 , ..., -0.7209007 , -0.2628672 , -0.49679   ]],

       [[ 2.794887  , -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]]], dtype=float32)

### Temperate diploid

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_diploid

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/temperate_diploid


In [ ]:
act_hold = []
act_hold2 = []
for counter in range(0,100):
  img = img_holder_temp_dipl[counter];
  x, = first(dls.test_dl([img]));
  with Hook(learn_inf.model[1]) as hook:
      with torch.no_grad(): output = learn_inf.model.eval()(x.cuda())
      act = hook.stored
#  cls = 1
  with HookBwd(learn_inf.model[1][6]) as hookg:
      with Hook(learn_inf.model[1][6]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
#  print("act: \n", act[0])
  act_hold.append(act)
  act_hold2.append(act.cpu().detach().numpy())


In [ ]:
act_temp_dipl = np.asarray(act_hold2)

In [ ]:
act_temp_dipl

array([[[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  0.74480754, -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  1.4509704 , -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 ,  0.9980766 , -0.7372962 ]],

       ...,

       [[-0.6916634 , -0.5712142 , -0.29715648, ..., -0.7209007 ,  2.9287143 , -0.7372962 ]],

       [[-0.6916634 , -0.5712142 ,  0.05380786, ..., -0.7209007 ,  0.83738476, -0.5024262 ]],

       [[-0.6916634 ,  0.9705017 ,  1.843571  , ..., -0.7209007 ,  2.5917215 ,  0.11333281]]], dtype=float32)

### Neotropical racemose

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/neotropical_racemose

/content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/neotropical_racemose


In [ ]:
act_hold = []
act_hold2 = []
for counter in range(0,100):
  img = img_holder_trop_neo[counter];
  x, = first(dls.test_dl([img]));
  with Hook(learn_inf.model[1]) as hook:
      with torch.no_grad(): output = learn_inf.model.eval()(x.cuda())
      act = hook.stored
#  cls = 1
  with HookBwd(learn_inf.model[1][6]) as hookg:
      with Hook(learn_inf.model[1][6]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
#  print("act: \n", act[0])
  act_hold.append(act)
  act_hold2.append(act.cpu().detach().numpy())


In [ ]:
act_trop_neo = np.asarray(act_hold2)

In [ ]:
act_trop_neo

array([[[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]],

       [[-0.49877098, -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]],

       [[ 0.49912098, -0.5712142 , -0.5788494 , ..., -0.3682443 , -0.6570689 , -0.7372962 ]],

       ...,

       [[ 0.13872603, -0.5712142 , -0.5788494 , ..., -0.46036986, -0.6570689 , -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]],

       [[-0.6916634 , -0.5712142 , -0.5788494 , ..., -0.7209007 , -0.6570689 , -0.7372962 ]]], dtype=float32)

### Paleotropical racemose

In [ ]:
cd /content/drive/MyDrive/DL_images/temp_vs_trop/test_reserve/paleotropical_racemose

In [ ]:
act_hold = []
act_hold2 = []
for counter in range(0,100):
  img = img_holder_trop_pal[counter];
  x, = first(dls.test_dl([img]));
  with Hook(learn_inf.model[1]) as hook:
      with torch.no_grad(): output = learn_inf.model.eval()(x.cuda())
      act = hook.stored
#  cls = 1
  with HookBwd(learn_inf.model[1][6]) as hookg:
      with Hook(learn_inf.model[1][6]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
#  print("act: \n", act[0])
  act_hold.append(act)
  act_hold2.append(act.cpu().detach().numpy())


In [ ]:
act_trop_pal = np.asarray(act_hold2)

In [ ]:
act_trop_pal

# Here, we reshape the data and use IVIS to reduce the dimensionality of larger vectors into 2 dimensions


In [ ]:
act_temp_dipl.shape

(96, 1, 512)

In [ ]:
act_temp_race.shape

(77, 1, 512)

In [ ]:
act_trop_neo.shape

(65, 1, 512)

In [ ]:
act_trop_pal.shape

(97, 1, 512)

In [ ]:
temp_dipl = act_temp_dipl.reshape(96,512)

In [ ]:
temp_race = act_temp_race.reshape(77,512)

In [ ]:
trop_neo = act_trop_neo.reshape(65,512)

In [ ]:
trop_pal = act_trop_pal.reshape(97,512)

In [ ]:
temp_dipl_df = pd.DataFrame(temp_dipl)

In [ ]:
temp_race_df = pd.DataFrame(temp_race)

In [ ]:
trop_neo_df = pd.DataFrame(trop_neo)

In [ ]:
trop_pal_df = pd.DataFrame(trop_pal)

In [ ]:
master_df = pd.concat([temp_dipl_df, temp_race_df, trop_neo_df, trop_pal_df]).reset_index(drop=True)

In [ ]:
master_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,-0.691663,-0.571214,-0.207373,1.340048,1.954015,-0.675078,-0.582589,0.381872,1.286634,0.699695,1.374756,0.766746,-0.674187,0.326608,-0.709829,-0.608424,-0.531011,0.142737,1.099762,-0.683820,1.490188,-0.404739,-0.551825,-0.255449,-0.461177,0.351392,-0.608277,-0.711490,0.378350,1.208475,0.820407,-0.546186,-0.536626,0.011769,-0.725924,-0.211382,-0.022904,-0.699665,-0.517046,-0.603305,-0.123988,0.236163,0.092779,-0.617688,0.641486,-0.723102,2.283974,-0.551760,0.168843,0.980499,-0.652809,-0.544619,-0.650951,-0.658579,0.434955,-0.733179,-0.681287,-0.602275,-0.607568,-0.597836,-0.558044,-0.534876,-0.732652,-0.618439,-0.735209,1.531703,-0.508205,0.889477,-0.539841,0.252373,2.007172,0.031651,0.557965,0.459732,0.570652,2.187614,-0.657392,0.449223,-0.732346,1.955508,0.471164,0.185777,-0.600698,-0.680539,-0.738040,-0.463774,-0.502022,-0.603419,-0.090581,-0.677370,-0.578214,-0.649889,-0.527105,-0.522319,-0.616557,-0.007762,-0.510287,-0.738312,0.269299,-0.701719,-0.625709,-0.599418,-0.683182,-0.662242,-0.647780,0.320661,-0.538078,-0.543737,-0.584759,-0.707653,-0.605577,0.249033,-0.587679,0.457507,-0.563401,-0.702326,0.097214,1.758280,1.418625,-0.575805,-0.541944,-0.797628,-0.143004,-0.606424,0.562093,-0.591283,-0.435212,-0.657093,-0.693313,-0.646302,2.072609,-0.689524,-0.642532,-0.706580,-0.627746,-0.678331,1.521882,-0.629560,-0.677444,1.923184,-0.663528,-0.707004,-0.762577,0.835421,-0.020753,-0.644303,1.710964,-0.181550,-0.677417,-0.636038,0.316008,-0.695598,-0.526848,-0.561655,-0.731375,-0.661913,0.702050,-0.324993,1.679014,-0.694844,2.016887,0.549630,-0.337822,-0.677877,-0.637634,-0.682967,-0.605465,-0.602873,1.537931,-0.147733,-0.038270,-0.228613,-0.629842,1.067286,-0.663988,1.209888,-0.672027,-0.209592,-0.196240,-0.541295,-0.499293,-0.443351,-0.572873,-0.725955,-0.391651,-0.726898,-0.563114,0.921346,0.105150,-0.460119,-0.109854,-0.628251,-0.638119,-0.632286,1.212448,-0.703545,1.501910,-0.632530,-0.188799,0.592532,0.888583,-0.595548,-0.504917,0.476729,-0.668407,-0.604832,-0.718233,-0.687406,-0.758910,1.585373,-0.016108,0.322057,1.584

In [ ]:
from ivis import Ivis

In [ ]:
my_df = master_df.to_numpy()

In [ ]:
ivis_mod_strict = Ivis(model='maaten', n_epochs_without_progress=100, k =100, batch_size = 128)
ivis_mod_strict.fit(my_df)


In [ ]:
import matplotlib.pyplot as plt

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [ ]:
embeddings = ivis_mod_strict.transform(my_df)


3/3 [==============================] - 0s 5ms/step


In [ ]:
print(embeddings)

[[ 1.6157197e+00  3.8209958e+00]
 [ 3.3226821e+00  4.1602707e+00]
 [ 1.8879910e+00  4.3926363e+00]
 [ 3.4051902e+00  4.4075990e+00]
 [ 1.3390762e+00  4.0415630e+00]
 [-5.3071332e-01  2.8477182e+00]
 [ 3.1946099e-01  1.8536125e+00]
 [ 3.0216801e+00  4.0076313e+00]
 [ 2.9016879e+00  3.5615423e+00]
 [ 3.0934722e+00  4.2313380e+00]
 [ 1.5464381e+00  3.9137497e+00]
 [ 1.2102177e+00  3.5201797e+00]
 [ 3.1486535e+00  4.8090148e+00]
 [ 1.2678665e+00  3.6821330e+00]
 [ 2.8152635e+00  4.2329588e+00]
 [ 6.3067716e-01  3.6788132e+00]
 [ 2.9625177e+00  4.2523193e+00]
 [ 2.3159602e+00  3.5027945e+00]
 [ 2.4553908e-01  3.0484433e+00]
 [ 1.8434491e+00  4.0831170e+00]
 [ 1.6157109e+00  3.4505653e+00]
 [ 9.6052814e-01  3.9775534e+00]
 [-4.8103482e-01  2.1832571e+00]
 [-2.9834203e-02  2.4821277e+00]
 [ 1.0319201e+00  3.9111946e+00]
 [ 2.1540601e+00  3.6408820e+00]
 [ 2.8450415e-01  2.1698530e+00]
 [ 2.1070483e+00  4.5015473e+00]
 [ 2.1085351e+00  4.2169056e+00]
 [ 3.4138215e+00  3.8705599e+00]
 [-9.56299

In [ ]:
embed = pd.DataFrame(embeddings)

# These embeddings can be used to make a 2-dimensional representation of morphospace

In [ ]:
embed

,0,1
0,1.615720,3.820996
1,3.322682,4.160271
2,1.887991,4.392636
3,3.405190,4.407599
4,1.339076,4.041563
...,...,...
330,2.963007,-1.473710
331,2.301740,-0.142881
332,2.816552,-0.905747
333,1.049766,-0.590789
